In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 460kB 57.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import re
import collections
from konlpy.tag import Twitter, Kkma
import pickle
import collections
import random
import numpy as np
import os
import time
import argparse

import math
import copy

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

## Data Loader

## 1. load_embed

In [ ]:
def load_pretrained_vector(embed_path, load_path):
    with open(embed_path, 'r') as inp, open(load_path, 'w') as outp:
        pos_count = '30185'    # line count of the tsv file (as string)
        dimensions = '200'    # vector size (as string)
        outp.write(' '.join([pos_count, dimensions]) + '\n')
        for line in inp:
            words = line.strip().split()
            outp.write(' ')
            if "]" in [w for word in words for w in word]:
                line = line.strip().replace("]", "")
                words = line.strip().split()
                outp.write(' '.join(words) + '\n')
            elif "[" in [w for word in words for w in word]:
                line = line.strip().replace("[", "")
                words = line.strip().split()
                outp.write(' '.join(words))
            else:
                outp.write(' '.join(words))

def load_vec_file(filepath):
    """
    Load .vec file. Get pos_dict, pos_embedding_vector
    :param filepath: String, path of .vec file
    :return:
        pos_size, embedding_size, pos2idx, idx2pos, embedding_vec
    """
    i = 0
    with open(filepath, 'r') as fout:
        pos_list = list()
        embedding_list = list()
        for line in fout:
            line = line.strip()
            if i == 0:
                pos_size = int(line.split(" ")[0])
                embedding_size = int(line.split(" ")[1])
                i += 1
                continue
            vector_list = list()
            line_sp = line.split(" ")
            for j in range(len(line_sp)):
                if j == 0:
                    continue
                elif j == 1:
                    # print(line_sp[j])
                    pos_list.append(line_sp[j])
                else:
                    # print(line_sp[j])
                    vector_list.append(line_sp[j])
            embedding_list.append(vector_list)

    pos2idx = dict()
    for pos in pos_list:
        pos2idx[pos] = len(pos2idx)
    idx2pos = dict(zip(pos2idx.values(), pos2idx.keys()))
    embedding_vec = np.array(embedding_list, dtype=np.float32)

    return pos_size, embedding_size, pos2idx, idx2pos, embedding_vec

if __name__ == "__main__":
    embed_path = "./embed/ko.tsv"
    load_path = "./embed/vec.txt"

    print("Loading {}...".format(embed_path))
    load_pretrained_vector(embed_path, load_path)
    print("Saved Complete {} !! ".format(load_path))

    # pos_size, embedding_size, pos2idx, idx2pos, embedding_vec = load_vec_file(load_path)
    # print(pos_size)
    # print(embedding_size)
    # print(pos2idx)
    # print(idx2pos)
    # print(np.shape(embedding_vec))

Loading ./embed/ko.tsv...


FileNotFoundError: ignored

## 2. preprocess_util

In [ ]:
np.random.seed(100)

if not os.path.isdir('./data'):
    os.mkdir('./data')

def _save_pickle(path, data):
    # save pkl
    f = open(path, 'wb')
    pickle.dump(data, f)
    f.close()


def sentence2pos(train_text, tag):
    if tag == "kkma":
        analyzer = Kkma()
    elif tag == "twitter":
        analyzer = Twitter()

    sentences = list()
    for line in train_text:
        sentence = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', line)
        if sentence:
            sentence = analyzer.pos(sentence)
            sentences.append(sentence)

    pos_counter = [['UNK', -1]]  # 빈도수 문제로 word_dict에 없는 word를 처리하기 위함. unknown
    pos_counter.extend(collections.Counter([word[0] for words in sentences for word in words]).most_common())
    print(pos_counter)

    pos_list = list()
    for pos, _ in pos_counter:
        pos_list.append(pos)

    return sentences, pos_list


def load_sentence(path):
    sentences = list()
    with open(path, 'r') as fout:
        for line in fout:
            pos_line = list()
            for pos in line.split():
                pos_line.append(pos)
            sentences.append(pos_line)
    return sentences


def get_sentence_pos(input):
    # Separate sentences into pos(morphemes)
    # 문장을 형태소로 분리
    # tag : "kkma" or "twitter"
    print("Sentence to pos in progressing...")
    data, pos_list = sentence2pos(input, tag="twitter")
    print("poke_count: ", count)
    print(len(data))
    print(data[0])
    print("pos_list: ", pos_list)

    # save pkl
    _save_pickle('./data/1_pk_real_data.pkl', data)
    _save_pickle('./data/pk_pos_list.pkl', pos_list)

    # save txt
    f = open('./data/1_pk_real_data.txt', 'w')
    for token in data:
        for word in token:
            word = str(word) + ' '
            f.write(word)
        f.write('\n')
    f.close()


def get_preprocess_data(embedpath, data_pos_list):

    # embed text 파일 읽기
    print("Loading embedding vector...")
    i = 0
    with open(embedpath, 'r') as fout:
        embed_pos_list = list()
        embedding_list = list()

        for line in fout:
            line = line.strip()
            if i == 0:
                pos_size = int(line.split(" ")[0])
                embedding_size = int(line.split(" ")[1])
                i += 1
                continue
            vector_list = list()
            line_sp = line.split(" ")
            for j in range(len(line_sp)):
                if j == 0:
                    continue
                elif j == 1:
                    # print(line_sp[j])
                    embed_pos_list.append(line_sp[j])
                else:
                    # print(line_sp[j])
                    vector_list.append(line_sp[j])
            embedding_list.append(vector_list)

    # embed vector의  pos2idx, idx2pos, embedding_vec 만듬
    pos2idx = dict()
    for pos in embed_pos_list:
        pos2idx[pos] = len(pos2idx)
    idx2pos = dict(zip(pos2idx.values(), pos2idx.keys()))
    print(pos2idx)
    print(idx2pos)

    embedding_vec = np.array(embedding_list, dtype=np.float32)
    print("before embed: ", np.shape(embedding_vec))

    print("Create new embedding vector...")
    # 현재 데이터에 해당되는 embedding vector만 추출.
    exist_idx = list()
    nonexist_pos = list()
    for data in data_pos_list:
        if data in list(pos2idx.keys()):
            exist_idx.append(pos2idx[data])
        else:
            nonexist_pos.append(data)

    embedding_vec = embedding_vec[sorted(exist_idx)]
    print(sorted(exist_idx))
    print("after embed: ", np.shape(embedding_vec))

    # 현재 데이터에는 있지만 embedding vector에 없는 데이터는 무작위 vector로 embedding vector에 추가
    start_embed = np.random.randn(1, embedding_size)
    add_embed = np.random.randn(len(nonexist_pos), embedding_size)
    embedding_vec = np.concatenate([start_embed, embedding_vec, add_embed], axis=0)
    print(len(nonexist_pos))
    print("after embed2: ", np.shape(embedding_vec))

    # 현재 데이터와 새로 만들어진 embedding vector에 맞는 pos2idx, idx2pos 만듬
    pos2idx = dict()
    pos2idx["<start>"] = len(pos2idx)
    for idx in sorted(exist_idx):
        pos = idx2pos[idx]
        pos2idx[pos] = len(pos2idx)
    for pos in nonexist_pos:
        pos2idx[pos] = len(pos2idx)
    idx2pos = dict(zip(pos2idx.values(), pos2idx.keys()))

    pos_size = len(pos2idx)

    # save pkl
    _save_pickle('./data/pk_pos2idx.pkl', pos2idx)
    _save_pickle('./data/pk_idx2pos.pkl', idx2pos)
    _save_pickle('./data/pretrain_embedding_vec.pkl', embedding_vec)
    print("Save all data as pkl !!")

    return pos_size, embedding_size


def _pkl_loading_test():
    # load sentences separated by pos (pkl)
    a = open('./data/1_pk_real_data.pkl', 'rb')
    sents = pickle.load(a)

    # load pos_list (pkl)
    a = open('./data/pk_pos_list.pkl', 'rb')
    pos_list = pickle.load(a)

    # load pos2idx (pkl)
    a = open('./data/pk_pos2idx.pkl', 'rb')
    pos2idx = pickle.load(a)

    # load idx2pos (pkl)
    a = open('./data/pk_idx2pos.pkl', 'rb')
    idx2pos = pickle.load(a)

    # load embedding_vec (pkl)
    a = open('./data/pretrain_embedding_vec.pkl', 'rb')
    embedding_vec = pickle.load(a)

    print(sents)
    print(len(pos_list))
    print(pos2idx)
    print(idx2pos)
    print(np.shape(embedding_vec))


if __name__ == "__main__":

    DATA_PATH = "./data/"
    embed_path = "./embed/vec.txt"

    data = pd.read_csv(DATA_PATH + 'Crawling_Lyrics.csv')
    data.dropna(axis=0, inplace = True)
    song_lyrics = list(data['lyrics'])

    lyrics = []
    for song_lyric in song_lyrics:
      sentences = []
      for sent in song_lyric.split('\n'):
        if sent == '':
          continue
        sent = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', sent)
        if sent:
          sent = analyzer.pos(sent)
        sentences.append(sent)
      lyrics.append(sentences)
    print(lyrics)

    # sentence와 pos_list pkl 만듬
    # 이미 pkl 만들었으면 주석 처리, 처음 사용시 주석 해제
    print("Data Loading and indexing...")
    get_sentence_pos(input)

    # load sentences separated by pos (pkl)
    a = open('./data/1_pk_real_data.pkl', 'rb')
    sents = pickle.load(a)

    # load pos_list (pkl)
    a = open('./data/pk_pos_list.pkl', 'rb')
    pos_list = pickle.load(a)
    print(pos_list)

    # make embedding vector and etc
    print("Data preprocessing in progress..")
    pos_size, embedding_size = get_preprocess_data(embed_path, pos_list)
    print("pos_size: ", pos_size)
    print("embedding_size: ", embedding_size)

    print("#### test ####")
    _pkl_loading_test()

## 3. preprocess_data

In [ ]:
def _save_pickle(path, data):
    # save pkl
    f = open(path, 'wb')
    pickle.dump(data, f)
    f.close()


def _get_before_dataset():
  DATA_PATH = "./data/"

  data = pd.read_csv(DATA_PATH + 'Crawling_Lyrics.csv')
  data.dropna(axis=0, inplace = True)
  song_lyrics = list(data['lyrics'])

  lyrics = []
  for song_lyric in song_lyrics:
    sentences = []
    for sent in song_lyric.split('\n'):
      if sent == '':
        continue
      sent = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', sent)
      if sent:
        sent = analyzer.pos(sent)
      sentences.append(sent)
    lyrics.append(sentences)
  print(lyrics)

  _save_pickle("./data/pk_before_input_data.pkl", lyrics)


def create_sequence(seq_length, lyrics):
  data = list()
  for lyric in lyrics:
    for sent in lyric:
      seq_data = list()

      # seq_data 개수가 seq_length가 될때 까지          
      for i in range(seq_length):
        if i >= len(sent):
          seq_data.append(('UNK' , ''))
        else:
          seq_data.append(sent[i])
    data.append(seq_data)

  _save_pickle("./data/2_pk_preprocessed_data.pkl", data)

  f = open('./data/2_pk_preprocessed_data.txt', 'w')
  for tokens in data:
    for word in tokens:
      word = str(word) + ' '
      f.write(word)
    f.write('\n')
  f.close()

def data_to_index(dataset, pos2idx):
    idx_dataset = list()
    for sent in dataset:
        idx_sentence = list()
        for word in sent:
            idx_sentence.append(pos2idx[word[0]])
        idx_dataset.append(idx_sentence)

    _save_pickle("./data/3_pk_data_index.pkl", idx_dataset)

    # save pk_data_index.txt
    f = open('./data/3_pk_data_index.txt', 'w')
    for idx_sent in idx_dataset:
        for word in idx_sent:
            word = str(word) + ' '
            f.write(word)
        f.write('\n')
    f.close()

if __name__ == "__main__":
    seq_length = 30  # max 52
    analyzer = Twitter()

    # 이미 pkl 만들었으면 주석 처리, 처음 사용시 주석 해제
    _get_before_dataset()

    # load before dataset
    a = open("./data/pk_before_input_data.pkl", 'rb')
    lyrics = pickle.load(a)

    print("Create Sequence in a length of seq_length...")
    create_sequence(seq_length, lyrics)

    print("Complete Creating sequence !!")

    # load after dataset
    a = open("./data/2_pk_preprocessed_data.pkl", 'rb')
    dataset = pickle.load(a)

    # load pos to index
    a = open("./data/pk_pos2idx.pkl", 'rb')
    pos2idx = pickle.load(a)

    print("Replace Sequence to Index...")
    data_to_index(dataset, pos2idx)

    print("Complete Creating sequence to index !!")

## create word2vec

In [ ]:
def _save_pickle(path, data):
    # save pkl
    f = open(path, 'wb')
    pickle.dump(data, f)
    f.close()


# data 불러옴
print("Load real data ...")
a = open('./data/1_pk_real_data.pkl', 'rb')
sentences = pickle.load(a)

# pk_idx2pos.pkl
a = open('./data/pk_idx2pos.pkl', 'rb')
idx2pos = pickle.load(a)

# pk_pos2idx.pkl
a = open('./data/pk_pos2idx.pkl', 'rb')
pos2idx = pickle.load(a)

sentence_idx = []
for sentence in sentences:
    words = []
    for word in sentence:
        words.append(pos2idx[word[0]])
    sentence_idx.append(words)

sentences_words = []
for sen in sentence_idx:
    sentence = []
    for pos_idx in sen:
        sentence.append(idx2pos[pos_idx])
        sentences_words.append(sentence)

# word2vec 학습
print("Training word2vec ...")
model = Word2Vec(sentences_words, size=30, window=5,min_count=0, workers=4, iter=10, sg=1)

# word2vec 테스트
print("Test word2vec ...")
print(model.most_similar("불"))

# word2vec에 <start>, UNK 등 추가 후 numpy로 저장
key = list(pos2idx.keys())
w2v = []
for k in key:
    if k == '<start>' or k == 'UNK' or k == '후다':
        print(k)
        w2v.append(np.random.randn(30)*0.1)
    else:
        w2v.append(model.wv[k])

w2v=np.array(w2v)

_save_pickle('./data/pk_embedding_vec.pkl', w2v)

print("Save word2vec !")

# pk_embedding_vec.pkl

a = open('./data/pk_embedding_vec.pkl', 'rb')
w2v_load = pickle.load(a)

print(np.shape(w2v_load))
print(w2v_load)

## 4. Data Loader

In [ ]:
class Gen_Data_loader():
    def __init__(self, batch_size, sen_length):
        self.batch_size = batch_size
        self.token_stream = []
        self.sen_length = sen_length

    def create_batches(self, data_file):
        self.token_stream = []
        with open(data_file, 'r') as f:
            for line in f:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                if len(parse_line) == self.sen_length:
                    self.token_stream.append(parse_line)

        self.num_batch = int(len(self.token_stream) / self.batch_size)
        self.token_stream = self.token_stream[:self.num_batch * self.batch_size]
        self.sequence_batch = np.split(np.array(self.token_stream), self.num_batch, 0)
        self.pointer = 0

    def next_batch(self):
        ret = self.sequence_batch[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0

In [ ]:
class Dis_dataloader():
    def __init__(self, batch_size, sen_length):
        self.batch_size = batch_size
        self.sentences = np.array([])
        self.labels = np.array([])
        self.sen_length = sen_length

    def load_train_data(self, positive_file, negative_file):
        # Load data
        positive_examples = []
        negative_examples = []
        with open(positive_file)as fin:
            for line in fin:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                positive_examples.append(parse_line)
        with open(negative_file)as fin:
            for line in fin:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                if len(parse_line) == self.sen_length:
                    negative_examples.append(parse_line)
        self.sentences = np.array(positive_examples + negative_examples)

        # Generate labels
        positive_labels = [[0, 1] for _ in positive_examples]
        negative_labels = [[1, 0] for _ in negative_examples]
        self.labels = np.concatenate([positive_labels, negative_labels], 0)

        # Shuffle the data
        shuffle_indices = np.random.permutation(np.arange(len(self.labels)))
        self.sentences = self.sentences[shuffle_indices]
        self.labels = self.labels[shuffle_indices]

        # Split batches
        self.num_batch = int(len(self.labels) / self.batch_size)
        self.sentences = self.sentences[:self.num_batch * self.batch_size]
        self.labels = self.labels[:self.num_batch * self.batch_size]
        self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
        self.labels_batches = np.split(self.labels, self.num_batch, 0)

        self.pointer = 0


    def next_batch(self):
        ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0

## 5. Generator

In [ ]:
class Generator(nn.Module):
    """ Generator """

    def __init__(self, vocab_size, embedding_dim, hidden_dim, use_cuda):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_cuda = use_cuda
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.init_params()

    def forward(self, x):
        """
        Embeds input and applies LSTM on the input sequence.
        Inputs: x
            - x: (batch_size, seq_len), sequence of tokens generated by generator
        Outputs: out
            - out: (batch_size * seq_len, vocab_size), lstm output prediction
        """
        self.lstm.flatten_parameters()
        h0, c0 = self.init_hidden(x.size(0))
        emb = self.embed(x) # batch_size * seq_len * emb_dim 
        out, _ = self.lstm(emb, (h0, c0)) # out: batch_size * seq_len * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # (batch_size*seq_len) * vocab_size
        return out

    def step(self, x, h, c):
        """
        Embeds input and applies LSTM one token at a time (seq_len = 1).
        Inputs: x, h, c
            - x: (batch_size, 1), sequence of tokens generated by generator
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state
        Outputs: out, h, c
            - out: (batch_size, vocab_size), lstm output prediction
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state 
        """
        self.lstm.flatten_parameters()
        emb = self.embed(x) # batch_size * 1 * emb_dim
        out, (h, c) = self.lstm(emb, (h, c)) # out: batch_size * 1 * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # batch_size * vocab_size
        return out, h, c

    def init_hidden(self, batch_size):
        h = torch.zeros(1, batch_size, self.hidden_dim)
        c = torch.zeros(1, batch_size, self.hidden_dim)
        if self.use_cuda:
            h, c = h.cuda(), c.cuda()
        return h, c
    
    def init_params(self):
        for param in self.parameters():
            param.data.uniform_(-0.05, 0.05)

    def sample(self, batch_size, seq_len, x=None):
        """
        Samples the network and returns a batch of samples of length seq_len.
        Outputs: out
            - out: (batch_size * seq_len)
        """
        samples = []
        if x is None:
            h, c = self.init_hidden(batch_size)
            x = torch.zeros(batch_size, 1, dtype=torch.int64)
            if self.use_cuda:
                x = x.cuda()
            for _ in range(seq_len):
                out, h, c = self.step(x, h, c)
                prob = torch.exp(out)
                x = torch.multinomial(prob, 1)
                samples.append(x)
        else:
            h, c = self.init_hidden(x.size(0))
            given_len = x.size(1)
            lis = x.chunk(x.size(1), dim=1)
            for i in range(given_len):
                out, h, c = self.step(lis[i], h, c)
                samples.append(lis[i])
            prob = torch.exp(out)
            x = torch.multinomial(prob, 1)
            for _ in range(given_len, seq_len):
                samples.append(x)
                out, h, c = self.step(x, h, c)
                prob = torch.exp(out)
                x = torch.multinomial(prob, 1)
        out = torch.cat(samples, dim=1) # along the batch_size dimension
        return out

## 6. Discriminator

In [ ]:

# a = open('./data/pretrain_embedding_vec.pkl', 'rb')
# word_embedding_matrix = pickle.load(a)
# word_embedding_matrix = word_embedding_matrix.astype(np.float32)

# An alternative to tf.nn.rnn_cell._linear function, which has been removed in Tensorfow 1.0.1
# The highway layer is borrowed from https://github.com/mkroutikov/tf-lstm-char-cnn

class Discriminator(nn.Module):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    Highway architecture based on the pooled feature maps is added. Dropout is adopted.
    """

    def __init__(self, num_classes, vocab_size, embedding_dim, filter_sizes, num_filters, dropout_prob):
        super(Discriminator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_f, (f_size, embedding_dim)) for f_size, num_f in zip(filter_sizes, num_filters)
        ])
        self.highway = nn.Linear(sum(num_filters), sum(num_filters))
        self.dropout = nn.Dropout(p = dropout_prob)
        self.fc = nn.Linear(sum(num_filters), num_classes)

    def forward(self, x):
        """
        Inputs: x
            - x: (batch_size, seq_len)
        Outputs: out
            - out: (batch_size, num_classes)
        """
        emb = self.embed(x).unsqueeze(1) # batch_size, 1 * seq_len * emb_dim
        convs = [F.relu(conv(emb)).squeeze(3) for conv in self.convs] # [batch_size * num_filter * seq_len]
        pools = [F.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in convs] # [batch_size * num_filter]
        out = torch.cat(pools, 1)  # batch_size * sum(num_filters)
        highway = self.highway(out)
        transform = F.sigmoid(highway)
        out = transform * F.relu(highway) + (1. - transform) * out # sets C = 1 - T
        out = F.log_softmax(self.fc(self.dropout(out)), dim=1) # batch * num_classes
        return out

## target LSTM

In [ ]:

class TargetLSTM(nn.Module):
    """ Target LSTM """

    def __init__(self,  vocab_size, embedding_dim, hidden_dim, use_cuda):
        super(TargetLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_cuda = use_cuda
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.init_params()

    def forward(self, x):
        """
        Embeds input and applies LSTM on the input sequence.
        Inputs: x
            - x: (batch_size, seq_len), sequence of tokens generated by generator
        Outputs: out
            - out: (batch_size, vocab_size), lstm output prediction
        """
        self.lstm.flatten_parameters()
        h0, c0 = self.init_hidden(x.size(0))
        emb = self.embed(x) # batch_size * seq_len * emb_dim 
        out, _ = self.lstm(emb, (h0, c0)) # out: seq_len * batch_size * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # seq_len * batch_size * vocab_size
        return out

    def step(self, x, h, c):
        """
        Embeds input and applies LSTM one token at a time (seq_len = 1).
        Inputs: x, h, c
            - x: (batch_size, 1), sequence of tokens generated by generator
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state
        Outputs: out, h, c
            - out: (batch_size, 1, vocab_size), lstm output prediction
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state 
        """
        self.lstm.flatten_parameters()
        emb = self.embed(x) # batch_size * 1 * emb_dim
        out, (h, c) = self.lstm(emb, (h, c)) # out: batch_size * 1 * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # batch_size * vocab_size
        return out, h, c

    def init_hidden(self, batch_size):
        h = torch.zeros((1, batch_size, self.hidden_dim))
        c = torch.zeros((1, batch_size, self.hidden_dim))
        if self.use_cuda:
            h, c = h.cuda(), c.cuda()
        return h, c
    
    def init_params(self):
        for param in self.parameters():
            param.data.normal_(0, 1)

    def sample(self, batch_size, seq_len):
        """
        Samples the network and returns a batch of samples of length seq_len.
        Outputs: out
            - out: (batch_size * seq_len)
        """
        samples = []
        h, c = self.init_hidden(batch_size)
        x = torch.zeros(batch_size, 1, dtype=torch.int64)
        if self.use_cuda:
            x = x.cuda()
        for _ in range(seq_len):
            out, h, c = self.step(x, h, c)
            prob = torch.exp(out)
            x = torch.multinomial(prob, 1)
            samples.append(x)
        out = torch.cat(samples, dim=1) # along the batch_size dimension
        return out

## PGLoss

In [ ]:
class PGLoss(nn.Module):
    """
    Pseudo-loss that gives corresponding policy gradients (on calling .backward()) 
    for adversial training of Generator
    """

    def __init__(self):
        super(PGLoss, self).__init__()

    def forward(self, pred, target, reward):
        """
        Inputs: pred, target, reward
            - pred: (batch_size, seq_len), 
            - target : (batch_size, seq_len), 
            - reward : (batch_size, ), reward of each whole sentence
        """
        one_hot = torch.zeros(pred.size(), dtype=torch.uint8)
        if pred.is_cuda:
            one_hot = one_hot.cuda()
        one_hot.scatter_(1, target.data.view(-1, 1), 1)
        loss = torch.masked_select(pred, one_hot)
        loss = loss * reward.contiguous().view(-1)
        loss = -torch.sum(loss)
        return loss

## data inter

In [ ]:

class GenDataIter:
    """ Toy data iter to load digits """

    def __init__(self, data_file, batch_size):
        super(GenDataIter, self).__init__()
        self.batch_size = batch_size
        self.data_lis = self.read_file(data_file)
        self.data_num = len(self.data_lis)
        self.indices = range(self.data_num)
        self.num_batches = math.ceil(self.data_num / self.batch_size)
        self.idx = 0
        self.reset()

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def reset(self):
        self.idx = 0
        random.shuffle(self.data_lis)

    def next(self):
        if self.idx >= self.data_num:
            raise StopIteration
        index = self.indices[self.idx : self.idx + self.batch_size]
        d = [self.data_lis[i] for i in index]
        d = torch.tensor(d)

        # 0 is prepended to d as start symbol
        data = torch.cat([torch.zeros(len(index), 1, dtype=torch.int64), d], dim=1)
        target = torch.cat([d, torch.zeros(len(index), 1, dtype=torch.int64)], dim=1)
        
        self.idx += self.batch_size
        return data, target

    def read_file(self, data_file):
        with open(data_file, 'r') as f:
            lines = f.readlines()
        lis = []
        for line in lines:
            l = [int(s) for s in list(line.strip().split())]
            lis.append(l)
        return lis


class DisDataIter:
    """ Toy data iter to load digits """

    def __init__(self, real_data_file, fake_data_file, batch_size):
        super(DisDataIter, self).__init__()
        self.batch_size = batch_size
        real_data_lis = self.read_file(real_data_file)
        fake_data_lis = self.read_file(fake_data_file)
        self.data = real_data_lis + fake_data_lis
        self.labels = [1 for _ in range(len(real_data_lis))] +\
                        [0 for _ in range(len(fake_data_lis))]
        self.pairs = list(zip(self.data, self.labels))
        self.data_num = len(self.pairs)
        self.indices = range(self.data_num)
        self.num_batches = math.ceil(self.data_num / self.batch_size)
        self.idx = 0
        self.reset()

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def reset(self):
        self.idx = 0
        random.shuffle(self.pairs)

    def next(self):
        if self.idx >= self.data_num:
            raise StopIteration
        index = self.indices[self.idx : self.idx + self.batch_size]
        pairs = [self.pairs[i] for i in index]
        data = [p[0] for p in pairs]
        label = [p[1] for p in pairs]
        data = torch.tensor(data)
        label = torch.tensor(label)
        self.idx += self.batch_size
        return data, label

    def read_file(self, data_file):
        with open(data_file, 'r') as f:
            lines = f.readlines()
        lis = []
        for line in lines:
            l = [int(s) for s in list(line.strip().split())]
            lis.append(l) 
        return lis

## roll out

In [ ]:
class Rollout(object):
    """ Rollout Policy """

    def __init__(self, model, update_rate):
        self.ori_model = model
        self.own_model = copy.deepcopy(model)
        self.update_rate = update_rate

    def get_reward(self, x, num, discriminator):
        """
        Inputs: x, num, discriminator
            - x: (batch_size, seq_len) input data
            - num: rollout number
            - discriminator: discrimanator model
        """
        rewards = []
        batch_size = x.size(0)
        seq_len = x.size(1)
        for i in range(num):
            for l in range(1, seq_len):
                data = x[:, 0:l]
                samples = self.own_model.sample(batch_size, seq_len, data)
                pred = discriminator(samples)
                pred = pred.cpu().data[:,1].numpy()
                if i == 0:
                    rewards.append(pred)
                else:
                    rewards[l-1] += pred

            # for the last token
            pred = discriminator(x)
            pred = pred.cpu().data[:, 1].numpy()
            if i == 0:
                rewards.append(pred)
            else:
                rewards[seq_len-1] += pred
        rewards = np.transpose(np.array(rewards)) / (1.0 * num) # batch_size * seq_len
        return rewards

    def update_params(self):
        dic = {}
        for name, param in self.ori_model.named_parameters():
            dic[name] = param.data
        for name, param in self.own_model.named_parameters():
            if name.startswith('emb'):
                param.data = dic[name]
            else:
                param.data = self.update_rate * param.data + (1 - self.update_rate) * dic[name]

## 4. sequence_gan

In [ ]:
# Arguemnts
parser = argparse.ArgumentParser(description='SeqGAN')
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument('--hpc', action='store_true', default=False,
                    help='set to hpc mode')
parser.add_argument('--data_path', type=str, default='/content/data/', metavar='PATH',
                    help='data path to save files (default: /content/data/)')
parser.add_argument('--rounds', type=int, default=150, metavar='N',
                    help='rounds of adversarial training (default: 150)')
parser.add_argument('--g_pretrain_steps', type=int, default=120, metavar='N',
                    help='steps of pre-training of generators (default: 120)')
parser.add_argument('--d_pretrain_steps', type=int, default=50, metavar='N',
                    help='steps of pre-training of discriminators (default: 50)')
parser.add_argument('--g_steps', type=int, default=1, metavar='N',
                    help='steps of generator updates in one round of adverarial training (default: 1)')
parser.add_argument('--d_steps', type=int, default=3, metavar='N',
                    help='steps of discriminator updates in one round of adverarial training (default: 3)')
parser.add_argument('--gk_epochs', type=int, default=1, metavar='N',
                    help='epochs of generator updates in one step of generate update (default: 1)')
parser.add_argument('--dk_epochs', type=int, default=3, metavar='N',
                    help='epochs of discriminator updates in one step of discriminator update (default: 3)')
parser.add_argument('--update_rate', type=float, default=0.8, metavar='UR',
                    help='update rate of roll-out model (default: 0.8)')
parser.add_argument('--n_rollout', type=int, default=16, metavar='N',
                    help='number of roll-out (default: 16)')
parser.add_argument('--vocab_size', type=int, default=10, metavar='N',
                    help='vocabulary size (default: 10)')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--n_samples', type=int, default=6400, metavar='N',
                    help='number of samples gerenated per time (default: 6400)')
parser.add_argument('--gen_lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate of generator optimizer (default: 1e-3)')
parser.add_argument('--dis_lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate of discriminator optimizer (default: 1e-3)')
parser.add_argument('--no_cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')


#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
g_embed_dim = 30 # embedding dimension (pretrained: 200, pk: 30)
g_hidden_dim = 300 # hidden state dimension of lstm cell
g_seq_len = 30 # sequence length
START_TOKEN = 0
PRE_EPOCH_NUM = 120  # supervise (maximum likelihood estimation) epochs
SEED = 88
BATCH_SIZE = 64

#########################################################################################
#  Discriminator  Hyper-parameters
#########################################################################################
d_num_class = 2
d_embed_dim = EMB_DIM
d_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30]
d_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
d_dropout_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64

#########################################################################################
#  Basic Training Parameters
#########################################################################################
TOTAL_BATCH = 200
generated_num = 100
sample_num = 10

# original seqgan parameter
# HIDDEN_DIM = 32
# PRE_EPOCH_NUM = 120
# TOTAL_BATCH = 200
# generated_num = 10000

POSITIVE_FILE = './data/3_pk_data_index.txt'
NEGATIVE_FILE = 'save/negative_sample.txt'
eval_file = 'save/eval_file.txt'
# "pretrain" or "poke"
embed_flag = "poke"

a = open('./data/3_pk_data_index.pkl', 'rb')
real_data = pickle.load(a)

a = open('./data/pk_pos2idx.pkl', 'rb')
vocab_to_int = pickle.load(a)

a = open('./data/pk_idx2pos.pkl', 'rb')
int_to_vocab = pickle.load(a)
print(int_to_vocab)

if embed_flag == "pretrain":
    a = open('./data/pretrain_embedding_vec.pkl', 'rb')
elif embed_flag == "poke":
    a = open('./data/pk_embedding_vec.pkl', 'rb')
word_embedding_matrix = pickle.load(a)
word_embedding_matrix = word_embedding_matrix.astype(np.float32)

# a = open('./data/word_dict.pickle', 'rb')
# word_dict = pickle.load(a)

real_data_vocab = [[int_to_vocab[i] for i in sample if int_to_vocab[i] != '<PAD>'] for sample in real_data]
real_data_vocab = [' '.join(sample) for sample in real_data_vocab]
print(len(real_data_vocab))


def generate_samples(model, batch_size, generated_num, output_file):
    samples = []
    for _ in range(int(generated_num / batch_size)):
        sample = model.sample(batch_size, g_seq_len).cpu().data.numpy().tolist()
        samples.extend(sample)
    with open(output_file, 'w') as fout:
        for sample in samples:
            string = ' '.join([str(s) for s in sample])
            fout.write('{}\n'.format(string))


def train_generator_MLE(gen, data_iter, criterion, optimizer, epochs, 
        gen_pretrain_train_loss, args):
    """
    Train generator with MLE
    """
    for epoch in range(epochs):
        total_loss = 0.
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = gen(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        data_iter.reset()
    avg_loss = total_loss / len(data_iter)
    print("Epoch {}, train loss: {:.5f}".format(epoch, avg_loss))
    gen_pretrain_train_loss.append(avg_loss)

def train_generator_PG(gen, dis, rollout, pg_loss, optimizer, epochs, args):
    """
    Train generator with the guidance of policy gradient
    """
    for epoch in range(epochs):
        # construct the input to the genrator, add zeros before samples and delete the last column
        samples = generator.sample(args.batch_size, g_seq_len)
        zeros = torch.zeros(args.batch_size, 1, dtype=torch.int64)
        if samples.is_cuda:
            zeros = zeros.cuda()
        inputs = torch.cat([zeros, samples.data], dim = 1)[:, :-1].contiguous()
        targets = samples.data.contiguous().view((-1,))

        # calculate the reward
        rewards = torch.tensor(rollout.get_reward(samples, args.n_rollout, dis))
        if args.cuda:
            rewards = rewards.cuda()

        # update generator
        output = gen(inputs)
        loss = pg_loss(output, targets, rewards)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def eval_generator(model, data_iter, criterion, args):
    """
    Evaluate generator with NLL
    """
    total_loss = 0.
    with torch.no_grad():
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            pred = model(data)
            loss = criterion(pred, target)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_iter)
    return avg_loss


def train_discriminator(dis, gen, criterion, optimizer, epochs, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args):
    """
    Train discriminator
    """
    generate_samples(gen, args.batch_size, args.n_samples, NEGATIVE_FILE)
    data_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    for epoch in range(epochs):
        correct = 0
        total_loss = 0.
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = dis(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            loss = criterion(output, target)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        data_iter.reset()
        avg_loss = total_loss / len(data_iter)
        acc = correct.item() / data_iter.data_num
        print("Epoch {}, train loss: {:.5f}, train acc: {:.3f}".format(epoch, avg_loss, acc))
        dis_adversarial_train_loss.append(avg_loss)
        dis_adversarial_train_acc.append(acc)


def eval_discriminator(model, data_iter, criterion, args):
    """
    Evaluate discriminator, dropout is enabled
    """
    correct = 0
    total_loss = 0.
    with torch.no_grad():
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = model(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            loss = criterion(output, target)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_iter)
    acc = correct.item() / data_iter.data_num
    return avg_loss, acc


def adversarial_train(gen, dis, rollout, pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args):
    """
    Adversarially train generator and discriminator
    """
    # train generator for g_steps
    print("#Train generator")
    for i in range(args.g_steps):
        print("##G-Step {}".format(i))
        train_generator_PG(gen, dis, rollout, pg_loss, gen_optimizer, args.gk_epochs, args)

    # train discriminator for d_steps
    print("#Train discriminator")
    for i in range(args.d_steps):
        print("##D-Step {}".format(i))
        train_discriminator(dis, gen, nll_loss, dis_optimizer, args.dk_epochs, 
            dis_adversarial_train_loss, dis_adversarial_train_acc, args)

    # update roll-out model
    rollout.update_params()


def make_sample(eval_file, sample_num):
    samples = []
    with open(eval_file, 'r') as f:
        for line in f:
            line = line.strip()
            line = line.split()
            parse_line = [x for x in line]
            samples.append(parse_line)

    sample_vocab = samples

    return sample_vocab


if __name__ == '__main__':
    # Parse arguments
    args = parser.parse_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)
    if not args.hpc:
        args.data_path = ''
    POSITIVE_FILE = args.data_path + POSITIVE_FILE
    NEGATIVE_FILE = args.data_path + NEGATIVE_FILE

    # Set models, criteria, optimizers
    generator = Generator(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)
    discriminator = Discriminator(d_num_class, args.vocab_size, d_embed_dim, d_filter_sizes, d_num_filters, d_dropout_prob)
    target_lstm = TargetLSTM(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)
    nll_loss = nn.NLLLoss()
    pg_loss = PGLoss()
    if args.cuda:
        generator = generator.cuda()
        discriminator = discriminator.cuda()
        target_lstm = target_lstm.cuda()
        nll_loss = nll_loss.cuda()
        pg_loss = pg_loss.cuda()
        cudnn.benchmark = True
    gen_optimizer = optim.Adam(params=generator.parameters(), lr=args.gen_lr)
    dis_optimizer = optim.SGD(params=discriminator.parameters(), lr=args.dis_lr)

    # Container of experiment data
    gen_pretrain_train_loss = []
    gen_pretrain_eval_loss = []
    dis_pretrain_train_loss = []
    dis_pretrain_train_acc = []
    dis_pretrain_eval_loss = []
    dis_pretrain_eval_acc = []
    gen_adversarial_eval_loss = []
    dis_adversarial_train_loss = []
    dis_adversarial_train_acc = []
    dis_adversarial_eval_loss = []
    dis_adversarial_eval_acc = []

    # Generate toy data using target LSTM
    print('#####################################################')
    print('Generating data ...')
    print('#####################################################\n\n')
    generate_samples(target_lstm, args.batch_size, args.n_samples, POSITIVE_FILE)

    # Pre-train generator using MLE
    print('#####################################################')
    print('Start pre-training generator with MLE...')
    print('#####################################################\n')
    gen_data_iter = GenDataIter(POSITIVE_FILE, args.batch_size)
    # for i in range(args.g_pretrain_steps):
    for i in range(1):      
        print("G-Step {}".format(i))
        train_generator_MLE(generator, gen_data_iter, nll_loss, 
            gen_optimizer, args.gk_epochs, 
            gen_pretrain_train_loss, args)
        generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
        eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
        gen_loss = eval_generator(target_lstm, eval_iter, nll_loss, args)
        gen_pretrain_eval_loss.append(gen_loss)
        print("eval loss: {:.5f}\n".format(gen_loss))
    print('#####################################################\n\n')

    # Pre-train discriminator
    print('#####################################################')
    print('Start pre-training discriminator...')
    print('#####################################################\n')
    for i in range(1):
    # for i in range(args.d_pretrain_steps):      
        print("D-Step {}".format(i))
        train_discriminator(discriminator, generator, nll_loss, 
            dis_optimizer, args.dk_epochs, 
            dis_adversarial_train_loss, dis_adversarial_train_acc, args)
        generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
        eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
        dis_loss, dis_acc = eval_discriminator(discriminator, eval_iter, nll_loss, args)
        dis_pretrain_eval_loss.append(dis_loss)
        dis_pretrain_eval_acc.append(dis_acc)
        print("eval loss: {:.5f}, eval acc: {:.3f}\n".format(dis_loss, dis_acc))
    print('#####################################################\n\n')

    # Adversarial training
    print('#####################################################')
    print('Start adversarial training...')
    print('#####################################################\n')
    rollout = Rollout(generator, args.update_rate)
    for i in range(1):    
    # for i in range(args.rounds):
        print("Round {}".format(i))
        adversarial_train(generator, discriminator, rollout, 
            pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
            dis_adversarial_train_loss, dis_adversarial_train_acc, args)
        generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
        gen_eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
        dis_eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
        gen_loss = eval_generator(target_lstm, gen_eval_iter, nll_loss, args)
        gen_adversarial_eval_loss.append(gen_loss)
        dis_loss, dis_acc = eval_discriminator(discriminator, dis_eval_iter, nll_loss, args)
        dis_adversarial_eval_loss.append(dis_loss)
        dis_adversarial_eval_acc.append(dis_acc)
        print("gen eval loss: {:.5f}, dis eval loss: {:.5f}, dis eval acc: {:.3f}\n"
            .format(gen_loss, dis_loss, dis_acc))

    # Save experiment data
    with open(args.data_path + 'experiment.pkl', 'wb') as f:
        pickle.dump(
            (gen_pretrain_train_loss,
                gen_pretrain_eval_loss,
                dis_pretrain_train_loss,
                dis_pretrain_train_acc,
                dis_pretrain_eval_loss,
                dis_pretrain_eval_acc,
                gen_adversarial_eval_loss,
                dis_adversarial_train_loss,
                dis_adversarial_train_acc,
                dis_adversarial_eval_loss,
                dis_adversarial_eval_acc),
            f,
            protocol=pickle.HIGHEST_PROTOCOL
        )


    samples = make_sample(eval_file, generated_num)
    samples = [[word for word in sample.split() if word != 'UNK'] for sample in samples]
    samples = [' '.join(sample) for sample in samples]

    f = open('./save/final_output_vocab.txt', 'w')
    for token in samples:
        token = token + '\n'
        f.write(token)
    f.close()



    a = open('./data/pk_embedding_vec.pkl', 'rb')
    w2v_load = pickle.load(a)




In [ ]:
samples = []
with open(eval_file, 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split()
        parse_line = [x for x in line]
        print(parse_line)

        for word in parse_line:
          if word in vocab_to_int.keys():
            words = vocab_to_int[word]
            samples.append(words)

sample_int = samples[:20]
sample_vocab = [int_to_vocab[sample] for sample in sample_int]
sample_vocab = [''.join(sample) for sample in sample_vocab]

In [ ]:
sample_int

In [ ]:
sample_vocab

In [ ]:
a = open('./data/pk_pos2idx.pkl', 'rb')
vocab_to_int = pickle.load(a)

vocab_to_int['나도']